In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
import re
import json

In [2]:
def show(x):
    print(x.shape)
    return x.head()

In [3]:
with open("../data/raw/science_parse_output.json") as file:
    lines = [json.loads(l) for l in tqdm(file.readlines())]
len(lines)

  0%|          | 0/1365 [00:00<?, ?it/s]

1365

In [4]:
lines[0]

{'name': '/home/jarobyte/Zotero/storage/FJGSUPN6/GCR-Powerpoint-Webinar-Jan-2020.pdf',
 'metadata': {'source': 'CRF',
  'title': None,
  'authors': ['Mimi McClure',
   'Laura Regassa',
   'Mike Roco',
   'Nora Savage',
   'Steven Breckler',
   'Toby Parcel'],
  'emails': ['gcrelig@nsf.gov.'],
  'sections': [{'heading': None,
    'text': 'NSF Big Idea: Growing Convergence Research\nNSF Big Idea: Growing Convergence Research\nWebinar Objectives\n• Overview of the program & program goals • Solicitation specifics • Review criteria • Q/A\nSubmit questions via the Q/A panel in Zoom or via email to gcrelig@nsf.gov.\nNSF Big Idea: Growing Convergence Research\nConvergence at NSF\n• Convergence is a multifaceted concept • It has long history at NSF; part of NSF’s portfolio since 1954 • It is currently an important component of several parts of NSF’s\nportfolio, including • the 6 research-oriented Big Ideas • NSF Convergence Accelerator • Growing Convergence Research (GCR) a process-oriented Big

In [5]:
raw = pd.DataFrame(lines)\
.assign(title = lambda df: df.metadata.map(lambda x: x["title"]),
        abstract = lambda df: df.metadata.map(lambda x: x["abstractText"]),
        sections = lambda df: df.metadata.map(lambda x: x["sections"]),
        references = lambda df: df.metadata.map(lambda x: x["references"]),
        citations = lambda df: df.metadata.map(lambda x: x["referenceMentions"]),
)\

show(raw)

(1365, 7)


,name,metadata,title,abstract,sections,references,citations
0,/home/jarobyte/Zotero/storage/FJGSUPN6/GCR-Pow...,"{'source': 'CRF', 'title': None, 'authors': ['...",None,None,"[{'heading': None, 'text': 'NSF Big Idea: Grow...",[],[]
1,/home/jarobyte/Zotero/storage/E47J4XWS/Beineke...,"{'source': 'CRF', 'title': 'Characterizations ...",Characterizations of Derived Graphs,The derived graph of a graph G has the edges o...,None,[{'title': 'On the Uniqueness of the Triangula...,"[{'referenceID': 0, 'context': 'JOURNAL OF COM..."
2,/home/jarobyte/Zotero/storage/K9SMVNFH/IDST at...,"{'source': 'META', 'title': 'IDST at TREC 2019...",IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"[{'heading': None, 'text': 'KEYWORDS cascade r...",[{'title': 'Overview of the TREC 2019 deep lea...,"[{'referenceID': 1, 'context': 'Our approach i..."
3,/home/jarobyte/Zotero/storage/FEFDDM7K/Kirsch ...,"{'source': 'CRF', 'title': 'BatchBALD: Efficie...",BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...","[{'heading': '1 Introduction', 'text': 'A key ...",[{'title': 'Toward the next generation of reco...,"[{'referenceID': 6, 'context': 'Active learnin..."
4,/home/jarobyte/Zotero/storage/GXE9CYPA/acl13-S...,"{'source': 'META', 'title': 'A Sentence Compre...",A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,"[{'heading': None, 'text': 'Proceedings of the...",[{'title': 'Syntax directed translations and t...,"[{'referenceID': 18, 'context': 'specific or n..."


In [6]:
raw_na = raw.where(raw.applymap(lambda x: x is not None and x != []))\
.dropna()\
.assign(    
    title_clean = lambda df: df.title.str.replace(r"\W", "", regex = True).str.lower()
)\
.applymap(json.dumps)\
.groupby("title_clean").min()\
.applymap(json.loads)\
.reset_index()\
.reset_index()\
.rename(columns = {"index":"paper_id"})


show(raw_na)

(1013, 9)


,paper_id,title_clean,name,metadata,title,abstract,sections,references,citations
0,0,"""abenchmarkandevaluationfortextextractionfrompdf""",/home/jarobyte/Zotero/storage/CUIW3UGB/Bast an...,"{'source': 'META', 'title': 'A Benchmark and E...",A Benchmark and Evaluation for Text Extraction...,Extracting the body text from a PDF document i...,"[{'heading': None, 'text': 'A Benchmark and Ev...",[{'title': 'A New Dataset for Fine-Grained Cit...,"[{'referenceID': 14, 'context': 'e DBLP datas..."
1,1,"""abenchmarkforsystematicgeneralizationinground...",/home/jarobyte/Zotero/storage/PS5BNMBF/Ruis et...,"{'source': 'CRF', 'title': 'A Benchmark for Sy...",A Benchmark for Systematic Generalization in G...,Humans easily interpret expressions that descr...,"[{'heading': '1 Introduction', 'text': 'Human ...",[{'title': 'Measuring compositionality in repr...,"[{'referenceID': 22, 'context': 'Modern deep n..."
2,2,"""ablationstudiesinartificialneuralnetworks""",/home/jarobyte/Zotero/storage/38P33CSH/Meyes e...,"{'source': 'CRF', 'title': 'Ablation Studies i...",Ablation Studies in Artificial Neural Networks,Ablation studies have been widely used in the ...,"[{'heading': '1 Introduction', 'text': 'Recent...",[{'title': 'Imagenet classification with deep ...,"[{'referenceID': 0, 'context': 'During the las..."
3,3,"""abriefintroductiontoweaklysupervisedlearning""",/home/jarobyte/Zotero/storage/QFN7J2WX/Zhou - ...,"{'source': 'META', 'title': 'A brief introduct...",A brief introduction to weakly supervised lear...,Supervised learning techniques construct predi...,"[{'heading': None, 'text': 'National Key Labor...","[{'title': 'Deep Learning', 'author': ['I Good...","[{'referenceID': 0, 'context': 'Most successfu..."
4,4,"""abstractiveeventsummarizationontwitter""",/home/jarobyte/Zotero/storage/PRWHRPAT/Li and ...,"{'source': 'CRF', 'title': 'Abstractive Event ...",Abstractive Event Summarization on Twitter,ive Event Summarization on Twitter Quanzhi Li ...,"[{'heading': 'CCS CONCEPTS', 'text': '• Comput...",[{'title': 'BERT: Pretraining of deep bidirect...,"[{'referenceID': 8, 'context': 'Our generation..."


In [7]:
papers = raw_na[["paper_id", "title", "abstract", "sections"]]\

show(papers)

(1013, 4)


,paper_id,title,abstract,sections
0,0,A Benchmark and Evaluation for Text Extraction...,Extracting the body text from a PDF document i...,"[{'heading': None, 'text': 'A Benchmark and Ev..."
1,1,A Benchmark for Systematic Generalization in G...,Humans easily interpret expressions that descr...,"[{'heading': '1 Introduction', 'text': 'Human ..."
2,2,Ablation Studies in Artificial Neural Networks,Ablation studies have been widely used in the ...,"[{'heading': '1 Introduction', 'text': 'Recent..."
3,3,A brief introduction to weakly supervised lear...,Supervised learning techniques construct predi...,"[{'heading': None, 'text': 'National Key Labor..."
4,4,Abstractive Event Summarization on Twitter,ive Event Summarization on Twitter Quanzhi Li ...,"[{'heading': 'CCS CONCEPTS', 'text': '• Comput..."


In [8]:
raw_na.citations.map(len, na_action = "ignore").sum()

51328

In [9]:
citations_raw = raw_na.citations.dropna().explode()
show(citations_raw)

(51328,)


0    {'referenceID': 14, 'context': 'e DBLP datase...
0    {'referenceID': 17, 'context': 'e Cora Inform...
0    {'referenceID': 0, 'context': 'e UMass Citati...
0    {'referenceID': 15, 'context': '[20] compiled ...
0    {'referenceID': 4, 'context': 'e CiteSeerX da...
Name: citations, dtype: object

In [10]:
citations = pd.DataFrame(citations_raw.tolist())\
.assign(paper_id = citations_raw.index)\
[["paper_id", "referenceID", "context", "startOffset", "endOffset"]]\
.rename(
    columns = {
        "referenceID":"internal_reference_id", 
        "startOffset":"start_offset", 
        "endOffset":"end_offset"
    }
)

show(citations)
citations.head(20)

(51328, 5)


,paper_id,internal_reference_id,context,start_offset,end_offset
0,0,14,e DBLP dataset [19] provides bibliographic me...,16,20
1,0,17,e Cora Information Extraction dataset [22] is...,39,43
2,0,0,e UMass Citation Field Extraction Dataset [1]...,43,46
3,0,15,[20] compiled a dataset consisting of metadata...,0,4
4,0,4,e CiteSeerX dataset [6] provides full bibliog...,21,24
5,0,21,e Grotoap dataset [27] consists of 113 articl...,19,23
6,0,22,"e follow-up dataset, Grotoap2 [28], consists ...",31,35
7,0,23,e data are extracted by a series of supervise...,98,102
8,0,2,e ACL Anthology Reference Corpus [4] provides...,34,37
9,0,13,pdohtml [18] converts a given PDF le to XML ...,9,13


In [11]:
citations.context[2]

'\x91e UMass Citation Field Extraction Dataset [1] consists of 1,829 manually labeled citation strings which originate from 1,200 articles from arXiv.'

In [12]:
references = raw_na.references.explode().reset_index()\
.assign(
    internal_reference_id = lambda df: df.groupby("index").cumcount(),
    title = lambda df: df.references.map(lambda x: x["title"], na_action = "ignore")
)\
.assign(
    title_clean = lambda df: df.title.str.replace(r"\W", "", regex = True).str.lower()
)\
.rename(columns = {"index":"paper_id"})


show(references)


(40032, 5)


,paper_id,references,internal_reference_id,title,title_clean
0,0,{'title': 'A New Dataset for Fine-Grained Cita...,0,A New Dataset for Fine-Grained Citation Field ...,anewdatasetforfinegrainedcitationfieldextraction
1,0,"{'title': 'PDFExtract', 'author': ['Ø.R. Berg'...",1,PDFExtract,pdfextract
2,0,{'title': 'e ACL Anthology Reference Corpus: ...,2,e ACL Anthology Reference Corpus: A Reference...,eaclanthologyreferencecorpusareferencedatasetf...
3,0,"{'title': 'LA-PdfText', 'author': ['G. Burns']...",3,LA-PdfText,lapdftext
4,0,"{'title': 'J', 'author': ['C. Caragea', 'J. Wu...",4,J,j


In [13]:
# In some papers, there are references for which science-parse didn't parse all the 
# citations correctly. These papers have more references than citations.
# There are two papers (181 and 640) that apparently had some trouble parsing the references.

(citations.groupby(["paper_id", "internal_reference_id"])\
 .size().groupby("paper_id").size()
 - references.groupby(["paper_id", "internal_reference_id"])\
 .size().groupby("paper_id").size())\
.sort_values(ascending = False)

paper_id
181     15
640      1
555      0
277      0
772      0
      ... 
103   -109
697   -140
820   -159
813   -173
395   -176
Length: 1013, dtype: int64

In [14]:
# The counts shown here are how many times a given paper appeared as reference in a paper, 
# parsed with science-parse

unique_references = references.groupby("title_clean")\
.agg(["min", "count"])\
.title\
.sort_values("count", ascending = False)\
.reset_index()\
.reset_index()\
.rename(columns = {"index":"reference_id", "min":"title", "count":"original_citations"})

show(unique_references)

(24701, 4)


,reference_id,title_clean,title,original_citations
0,0,bertpretrainingofdeepbidirectionaltransformers...,BERT: Pre-training of Deep Bidirectional Trans...,150
1,1,distributedrepresentationsofwordsandphrasesand...,Distributed Representations of Words and Phras...,147
2,2,attentionisallyouneed,Attention Is All You Need,136
3,3,gloveglobalvectorsforwordrepresentation,GloVe : Global Vectors for Word Representation,134
4,4,adamamethodforstochasticoptimization,Adam: A Method for Stochastic Optimization,111


In [15]:
# There are some rows for which there are discrepancies because science-parse failed to parse them correctly

full = citations.merge(
    references, 
    how = "left", 
    on = ["paper_id", "internal_reference_id"]
)\
.merge(unique_references.drop(columns = "title"), on = "title_clean", how = "left")

show(full)

(51328, 10)


,paper_id,internal_reference_id,context,start_offset,end_offset,references,title,title_clean,reference_id,original_citations
0,0,14,e DBLP dataset [19] provides bibliographic me...,16,20,"{'title': 'DBLP - Some Lessons Learned', 'auth...",DBLP - Some Lessons Learned,dblpsomelessonslearned,15388.0,1.0
1,0,17,e Cora Information Extraction dataset [22] is...,39,43,{'title': 'Automating the Construction of Inte...,Automating the Construction of Internet Portal...,automatingtheconstructionofinternetportalswith...,14739.0,1.0
2,0,0,e UMass Citation Field Extraction Dataset [1]...,43,46,{'title': 'A New Dataset for Fine-Grained Cita...,A New Dataset for Fine-Grained Citation Field ...,anewdatasetforfinegrainedcitationfieldextraction,18841.0,1.0
3,0,15,[20] compiled a dataset consisting of metadata...,0,4,{'title': 'Evaluation of Header Metadata Extra...,Evaluation of Header Metadata Extraction Appro...,evaluationofheadermetadataextractionapproaches...,22352.0,1.0
4,0,4,e CiteSeerX dataset [6] provides full bibliog...,21,24,"{'title': 'J', 'author': ['C. Caragea', 'J. Wu...",J,j,240.0,11.0


In [16]:
full.loc[full.title_clean.isna()]

,paper_id,internal_reference_id,context,start_offset,end_offset,references,title,title_clean,reference_id,original_citations
9475,181,82,"Furthermore, we observe that changing the scal...",87,91,NaN,NaN,NaN,NaN,NaN
9546,181,82,"[85] Kaiming He, Xiangyu Zhang, Shaoqing Ren, ...",0,4,NaN,NaN,NaN,NaN,NaN
9547,181,83,"[86] Razvan Pascanu, Tomas Mikolov, and Yoshua...",0,4,NaN,NaN,NaN,NaN,NaN
9548,181,84,"[87] Ekin D Cubuk, Barret Zoph, Jonathon Shlen...",0,4,NaN,NaN,NaN,NaN,NaN
9549,181,85,"[88] Christian Szegedy, Vincent Vanhoucke, Ser...",0,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
36887,697,505,"Of course, this idea isn’t exactly new either!...",47,68,NaN,NaN,NaN,NaN,NaN
36888,697,459,"Going much further back, Wilbur [2001] attempt...",25,39,NaN,NaN,NaN,NaN,NaN
36891,697,435,", 2009] or linear combinations [Vogt and Cottr...",31,56,NaN,NaN,NaN,NaN,NaN
36894,697,496,The idea was to filter the MS MARCO passage ra...,147,173,NaN,NaN,NaN,NaN,NaN


### just checking that everything makes sense

In [17]:
full.groupby(["reference_id", "paper_id"]).size()\
.sort_values(ascending = False)\
.rename("mentions")\
.reset_index()\
.groupby("reference_id").size()\
.sort_values(ascending = False)

reference_id
0.0        140
2.0        123
1.0        121
3.0        110
4.0         83
          ... 
10631.0      1
10634.0      1
10636.0      1
10638.0      1
24700.0      1
Length: 18983, dtype: int64

In [18]:
full.groupby("reference_id").size().sort_values(ascending = False)

reference_id
0.0        390
2.0        224
1.0        223
3.0        190
7.0        177
          ... 
11780.0      1
11782.0      1
11783.0      1
11787.0      1
24700.0      1
Length: 18983, dtype: int64

In [19]:
full.groupby(["title_clean", "paper_id"]).size()\
.sort_values(ascending = False)\
.rename("mentions")\
.reset_index()\
.groupby("title_clean").size()\
.sort_values(ascending = False)

title_clean
bertpretrainingofdeepbidirectionaltransformersforlanguageunderstanding    140
attentionisallyouneed                                                     123
distributedrepresentationsofwordsandphrasesandtheircompositionality       121
gloveglobalvectorsforwordrepresentation                                   110
adamamethodforstochasticoptimization                                       83
                                                                         ... 
fewshotobjectrecognitionfrommachinelabeledwebimages                         1
fewshottextclassificationwithdistributionalsignatures                       1
fewshottextclassificationwithinductionnetwork                               1
fieldextractionbyhybridincrementalandaprioristructuraltemplates             1
рашеваирбожинователепсихологиятачастотелектроннотоздравеопазванеакад        1
Length: 18983, dtype: int64

In [20]:
full.groupby("title_clean").size().sort_values(ascending = False)

title_clean
bertpretrainingofdeepbidirectionaltransformersforlanguageunderstanding                              390
attentionisallyouneed                                                                               224
distributedrepresentationsofwordsandphrasesandtheircompositionality                                 223
gloveglobalvectorsforwordrepresentation                                                             190
efficientestimationofwordrepresentationsinvectorspace                                               177
                                                                                                   ... 
howtodrawadirectedgraph                                                                               1
howtoexplainindividualclassificationdecisions                                                         1
howtofinetunebertfortextclassificationinccl                                                           1
howtofinetunebertfortextclassificationinchinanationa

# final datasets

In [21]:
final_citations = full[
    [
        "paper_id", 
         "internal_reference_id", 
         "reference_id", 
         "context", 
         "start_offset", 
         "end_offset"
    ]
]\
.dropna()\
.assign(
    reference_id = lambda df: df.reference_id.map(
        int, 
        na_action = "ignore"
    )
)
show(final_citations)

(51200, 6)


,paper_id,internal_reference_id,reference_id,context,start_offset,end_offset
0,0,14,15388,e DBLP dataset [19] provides bibliographic me...,16,20
1,0,17,14739,e Cora Information Extraction dataset [22] is...,39,43
2,0,0,18841,e UMass Citation Field Extraction Dataset [1]...,43,46
3,0,15,22352,[20] compiled a dataset consisting of metadata...,0,4
4,0,4,240,e CiteSeerX dataset [6] provides full bibliog...,21,24


In [22]:
final_papers = papers.assign(
    text = lambda df: df.sections.map(
        lambda x: " ".join([s["text"] for s in x]), 
        na_action = "ignore"
    )
)\
.assign(title = lambda df: df.title.str.replace(r"\s+", " ", regex = True).str.strip(),
        abstract = lambda df: df.abstract.str.replace(
            r"\s+", " ", 
            regex = True
        ).str.strip(),
        text = lambda df: df.text.str.replace(r"\s+", " ", regex = True).str.strip())
show(final_papers)

(1013, 5)


,paper_id,title,abstract,sections,text
0,0,A Benchmark and Evaluation for Text Extraction...,Extracting the body text from a PDF document i...,"[{'heading': None, 'text': 'A Benchmark and Ev...",A Benchmark and Evaluation for Text Extraction...
1,1,A Benchmark for Systematic Generalization in G...,Humans easily interpret expressions that descr...,"[{'heading': '1 Introduction', 'text': 'Human ...",Human language is a fabulous tool for generali...
2,2,Ablation Studies in Artificial Neural Networks,Ablation studies have been widely used in the ...,"[{'heading': '1 Introduction', 'text': 'Recent...",Recent research on deep learning (DL) has brou...
3,3,A brief introduction to weakly supervised lear...,Supervised learning techniques construct predi...,"[{'heading': None, 'text': 'National Key Labor...",National Key Laboratory for Novel Software Tec...
4,4,Abstractive Event Summarization on Twitter,ive Event Summarization on Twitter Quanzhi Li ...,"[{'heading': 'CCS CONCEPTS', 'text': '• Comput...",• Computing methodologies → Natural language p...


In [23]:
references_abstract = unique_references\
.merge(
    final_papers.assign(
        title_clean = lambda df: df.title.str.replace(r"\W", "", regex = True).str.lower()
    ),
    how = "left"
)\
[["reference_id", "title", "abstract", "original_citations", "title_clean"]]\
.drop_duplicates()

show(references_abstract)

(24701, 5)


,reference_id,title,abstract,original_citations,title_clean
0,0,BERT: Pre-training of Deep Bidirectional Trans...,We introduce a new language representation mod...,150,bertpretrainingofdeepbidirectionaltransformers...
1,1,Distributed Representations of Words and Phras...,NaN,147,distributedrepresentationsofwordsandphrasesand...
2,2,Attention Is All You Need,The dominant sequence transduction models are ...,136,attentionisallyouneed
3,3,GloVe : Global Vectors for Word Representation,NaN,134,gloveglobalvectorsforwordrepresentation
4,4,Adam: A Method for Stochastic Optimization,NaN,111,adamamethodforstochasticoptimization


# complete references from the arxiv dataset

In [24]:
titles = []
abstracts = []
with open("../data/raw/arxiv-metadata-oai-snapshot.json") as file:
    lines = file.readlines()
    for l in tqdm(lines):
        d = json.loads(l)
        titles.append(re.sub(r"\W", "", d["title"]).lower())
        abstracts.append(d["abstract"])

  0%|          | 0/2028910 [00:00<?, ?it/s]

In [25]:
arxiv = pd.DataFrame(dict(title_clean = titles, abstract = abstracts))
show(arxiv)

(2028910, 2)


,title_clean,abstract
0,calculationofpromptdiphotonproductioncrosssect...,A fully differential calculation in perturba...
1,sparsitycertifyinggraphdecompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,theevolutionoftheearthmoonsystembasedonthedark...,The evolution of Earth-Moon system is descri...
3,adeterminantofstirlingcyclenumberscountsunlabe...,We show that a determinant of Stirling cycle...
4,fromdyadiclambda_alphatolambda_alpha,In this paper we show how to compute the $\L...


In [26]:
full_references = references_abstract.merge(arxiv, on = "title_clean", how = "left")\
.assign(abstract = lambda df: df.abstract_y.combine_first(df.abstract_x))\
.drop(columns = ["abstract_x", "title_clean", "abstract_y"])\
.drop_duplicates()\
.assign(
    title = lambda df: df.title.str.replace(
        r"\s+", 
        " ", 
        regex = True
    ).str.strip(),
    abstract = lambda df: df.abstract.str.replace(
        r"\s+", " ", 
        regex = True
    ).str.strip()
)\
.groupby("reference_id").min()\
.reset_index()

show(full_references)
full_references.head(20)

(24701, 4)


,reference_id,title,original_citations,abstract
0,0,BERT: Pre-training of Deep Bidirectional Trans...,150,We introduce a new language representation mod...
1,1,Distributed Representations of Words and Phras...,147,The recently introduced continuous Skip-gram m...
2,2,Attention Is All You Need,136,The dominant sequence transduction models are ...
3,3,GloVe : Global Vectors for Word Representation,134,NaN
4,4,Adam: A Method for Stochastic Optimization,111,"We introduce Adam, an algorithm for first-orde..."
5,5,Long Short-Term Memory,88,NaN
6,6,Neural Machine Translation By Jointly Learning...,84,Neural machine translation is a recently propo...
7,7,Efficient Estimation of Word Representations i...,84,We propose two novel model architectures for c...
8,8,Deep Contextualized Word Representations,81,We introduce a new type of deep contextualized...
9,9,Deep Residual Learning for Image Recognition,69,Deeper neural networks are more difficult to t...


# paper sections

In [27]:
paper_sections = final_papers\
.drop(columns = ["title", "text", "abstract"])\
.explode("sections")\
.assign(
    section_order = lambda df: df.groupby("paper_id").cumcount(),
    heading = lambda df: df.sections.map(lambda x: x["heading"], na_action = "ignore"),
    text = lambda df: df.sections.map(lambda x: x["text"], na_action = "ignore"))\
.drop(columns = "sections")\
.assign(heading = lambda df: df.heading.str.replace(r"\s+", " ", regex = True).str.strip(),
        text = lambda df: df.text.str.replace(r"\s+", " ", regex = True).str.strip())

show(paper_sections)

(18010, 4)


,paper_id,section_order,heading,text
0,0,0,None,A Benchmark and Evaluation for Text Extraction...
0,0,1,1 INTRODUCTION,PDF continues to be one of the most popular el...
0,0,2,1.1 Kinds of semantic information,"In the following, we brie y describe the kind ..."
0,0,3,1.2 Existing tools,A large number of tools for text extraction fr...
0,0,4,1.3 Contributions,is paper is about an extensive evaluation of e...


# writing to disk

In [28]:
final_citations.to_csv("../data/citations.csv", index = False)

In [29]:
final_papers.drop(columns = "sections").to_csv(
    "../data/papers.csv", 
    errors = "replace", 
    index = False
)

In [30]:
full_references.to_csv("../data/references.csv", index = False)

In [31]:
paper_sections.to_csv("../data/paper_sections.csv", index = False, errors = "replace")

# queries

In [32]:
unique_references.query("reference_id == 790")

,reference_id,title_clean,title,original_citations
790,790,knowledgederivedfromwikipediaforcomputingseman...,Knowledge Derived from Wikipedia for Computing...,5


In [33]:
references_abstract.query("reference_id == 2638")

,reference_id,title,abstract,original_citations,title_clean
2638,2638,Solving Rubik’s Cube with a robot hand,NaN,2,solvingrubikscubewitharobothand


In [34]:
full_references.query("reference_id == 790").abstract.tolist()

[nan]

In [35]:
arxiv.query("title_clean == 'overviewofthetrec2019deeplearningtrack'")

,title_clean,abstract
1258175,overviewofthetrec2019deeplearningtrack,The Deep Learning Track is a new track for T...


In [36]:
arxiv.query("title_clean == 'overviewofthetrec2020deeplearningtrack'")

,title_clean,abstract
1424151,overviewofthetrec2020deeplearningtrack,This is the second year of the TREC Deep Lea...
